In [1]:
query =  '''
"proc sql;
	create table Wenlei.DW_Example_latest AS
	SELECT DISTINCT 
			T2.POLICY_ID,
			T2.POLICY,
			strip(left(T2.ROLE)),
            strip(left(T2.type)),			
			T2.SEQ_ID, 
			INTCK('day', T1.LastPayDate, today()) as DayPaid,
			(today()-T1.date_created) as DaysBtw			
	FROM 	WORK.DW_Table1 t1 
			INNER JOIN
			WORK.DW_Table2 t2 ON (t1.ID = t2.ID  AND T1.POLICY_ID=T2.POLICY_ID 
										   AND t1.MAX_PROCESS_Time=t2.PROCESS_Time)
	WHERE t2.ID<>-1 
	;
quit;"
'''

In [2]:
#goal strip(left(b.type_rw)) -> trim(b.type_rw) 
import re
text = '''strip(left(b.type_rw)) abc'''
pattern = re.compile("(strip\(left\().+\)\)", re.IGNORECASE)
match = pattern.search(text)
match.group(0)
# for match in pattern.finditer(text):
#     print (match.group(1), match.group(2))


'strip(left(b.type_rw))'

In [3]:
#replace prefix
def remove_prefix(text, prefix):
    return text[len(prefix):] if text.startswith(prefix) else text
#remove appendix
def remove_appendix(text, appendix):
    return text[:len(text)-len(appendix)] if text.endswith(appendix) else text

In [4]:
stringX  = match.group(0)
string_kept = remove_appendix (stringX, '))')
#print(string_kept)
string_kept = remove_prefix (string_kept, 'strip(left(')
#print(string_kept)
string_kept = 'trim(' + string_kept + ')'
text1 = text.replace(stringX, string_kept )
text1


'trim(b.type_rw) abc'

In [5]:
'''
https://stackoverflow.com/questions/6930982/how-to-use-a-variable-inside-a-regular-expression
https://docs.python.org/3/howto/regex.html#the-backslash-plague
'''
#define function to replace both side of string
def replace_both_sides (text, prefix, appendix, new_prefix, new_appendix):
    import re
    pattern = re.compile((re.escape(prefix)) + r'.+' + re.escape(appendix) , re.IGNORECASE)    #left side, keep group, use escape to remove all \
    if pattern.search(text) != None:
        match = pattern.search(text)  #use search for any place in a row
       # return match.group(0)
        stringX  = match.group(0)  # get matched
        string_kept = remove_appendix (stringX, appendix) #remove appendix
        #print(string_kept) 
        string_kept = remove_prefix (string_kept, prefix) #remove prefix
        #print(string_kept)
        string_kept = new_prefix + string_kept + new_appendix  #form replacement str
        text1 = text.replace(stringX, string_kept )  #replace string
        return text1
    else:
        return text



In [7]:
#test run
print(replace_both_sides (r'strip(left(b.type_rw)) abc', r"strip(left(", r"))", r'trim(', r')'))
print(replace_both_sides (r'strip(left(c.type_rw)) abc', r"strip(left(", r"))", r'trim(', r')'))


trim(b.type_rw) abc
trim(c.type_rw) abc


In [11]:
query_m= replace_both_sides (query, r"strip(left(", r"))", r'trim(', r')')
print(query_m)


"proc sql;
	create table Wenlei.DW_Example_latest AS
	SELECT DISTINCT 
			T2.POLICY_ID,
			T2.POLICY,
			trim(T2.ROLE),
            strip(left(T2.type)),			
			T2.SEQ_ID, 
			INTCK('day', T1.LastPayDate, today()) as DayPaid,
			(today()-T1.date_created) as DaysBtw			
	FROM 	WORK.DW_Table1 t1 
			INNER JOIN
			WORK.DW_Table2 t2 ON (t1.ID = t2.ID  AND T1.POLICY_ID=T2.POLICY_ID 
										   AND t1.MAX_PROCESS_Time=t2.PROCESS_Time)
	WHERE t2.ID<>-1 
	;
quit;"



In [12]:
#https://stackoverflow.com/questions/12870178/looping-through-python-regex-matches
import re
text =  query
prefix = r"strip(left("
appendix = r"))"
new_prefix =  r'trim('
new_appendix =r')'
pattern = re.compile((re.escape(prefix)) + r'.+' + re.escape(appendix) , re.IGNORECASE)    #left side, keep group, use escape to remove all \
matches = re.findall(pattern, text)  #use findall instead of search use loop to get the value
for g1 in matches:
    print (g1)


strip(left(T2.ROLE))
strip(left(T2.type))


In [13]:
def replace_both_sides_All (text, prefix, appendix, new_prefix, new_appendix):
    import re
    pattern = re.compile((re.escape(prefix)) + r'.+' + re.escape(appendix) , re.IGNORECASE)    #left side, keep group, use escape to remove all \
    if pattern.search(text) != None:
        matches = re.findall(pattern, text)  #use search for any place in a row
        for match in matches:  #loop through all match
           # return match.group(0)
            stringX  = match  # get matched
            string_kept = remove_appendix (stringX, appendix) #remove appendix
            #print(string_kept) 
            string_kept = remove_prefix (string_kept, prefix) #remove prefix
            #print(string_kept)
            string_kept = new_prefix + string_kept + new_appendix  #form replacement str
            text = text.replace(stringX, string_kept )  #replace string
        return text      
    else:
        return text

In [14]:
print(replace_both_sides_All (query, r"strip(left(", r"))", r'trim(', r')'))


"proc sql;
	create table Wenlei.DW_Example_latest AS
	SELECT DISTINCT 
			T2.POLICY_ID,
			T2.POLICY,
			trim(T2.ROLE),
            trim(T2.type),			
			T2.SEQ_ID, 
			INTCK('day', T1.LastPayDate, today()) as DayPaid,
			(today()-T1.date_created) as DaysBtw			
	FROM 	WORK.DW_Table1 t1 
			INNER JOIN
			WORK.DW_Table2 t2 ON (t1.ID = t2.ID  AND T1.POLICY_ID=T2.POLICY_ID 
										   AND t1.MAX_PROCESS_Time=t2.PROCESS_Time)
	WHERE t2.ID<>-1 
	;
quit;"



In [24]:
#use Re to solve this issue more elegantly
prefix=  r"strip(left("
appendix =r"))"
pattern = re.compile((re.escape(prefix)) + r'(.+)' + re.escape(appendix) , re.IGNORECASE) 
print(re.sub(pattern, r'trim(\1)', query))


"proc sql;
	create table Wenlei.DW_Example_latest AS
	SELECT DISTINCT 
			T2.POLICY_ID,
			T2.POLICY,
			trim(T2.ROLE),
            trim(T2.type),			
			T2.SEQ_ID, 
			INTCK('day', T1.LastPayDate, today()) as DayPaid,
			(today()-T1.date_created) as DaysBtw			
	FROM 	WORK.DW_Table1 t1 
			INNER JOIN
			WORK.DW_Table2 t2 ON (t1.ID = t2.ID  AND T1.POLICY_ID=T2.POLICY_ID 
										   AND t1.MAX_PROCESS_Time=t2.PROCESS_Time)
	WHERE t2.ID<>-1 
	;
quit;"



In [25]:
print(re.sub(pattern, r'trim(\1)', 'strip(left(T2.type))'))

trim(T2.type)


In [22]:
#trouble shooting in regex101
# https://stackoverflow.com/questions/47966350/regex-to-include-all-characters-with-at-least-one-letter-and-at-least-6-characte
# https://stackoverflow.com/questions/3028642/regular-expression-for-letters-numbers-and
#test another pattern
prefix=  r"strip(left("
appendix =r"))"
pattern = re.compile((re.escape(prefix)) + r'([a-zA-Z0-9_.-]*)' + re.escape(appendix) , re.IGNORECASE) 
print(re.sub(pattern, r'trim(\1)', query))


"proc sql;
	create table Wenlei.DW_Example_latest AS
	SELECT DISTINCT 
			T2.POLICY_ID,
			T2.POLICY,
			trim(T2.ROLE),
            trim(T2.type),			
			T2.SEQ_ID, 
			INTCK('day', T1.LastPayDate, today()) as DayPaid,
			(today()-T1.date_created) as DaysBtw			
	FROM 	WORK.DW_Table1 t1 
			INNER JOIN
			WORK.DW_Table2 t2 ON (t1.ID = t2.ID  AND T1.POLICY_ID=T2.POLICY_ID 
										   AND t1.MAX_PROCESS_Time=t2.PROCESS_Time)
	WHERE t2.ID<>-1 
	;
quit;"

